In [3]:
import pandas as pd
import geopandas as gpd
import numpy as np
import folium
from folium import Marker
from geopandas.tools import geocode

In [2]:
def embed_map(m, file_name):
    """Generates an HTML map and displays it"""
    
    from IPython.display import IFrame
    m.save(file_name)
    return IFrame(file_name, width='100%', height='500px')

In [4]:
result = geocode("The Great Pyramid of Giza", provider="nominatim")
result

,geometry,address
0,POINT (31.13424 29.97913),"هرم خوفو, Cause way, كوم الأخضر, الجيزة, محافظ..."


In [5]:
point = result.geometry.iloc[0]
print("Latitude: ", point.y)
print("Longitude: ", point.x)

Latitude:  29.9791264
Longitude:  31.1342383751015


In [6]:
%%bash
cp /Users/gurupratap.matharu/Downloads/geospatial-learn-course-data/top_universities.csv ../datasets/


In [18]:
universities = pd.read_csv("../datasets/top_universities.csv")
universities.head()

,Name
0,University of Oxford
1,University of Cambridge
2,Imperial College London
3,ETH Zurich
4,UCL


In [22]:
universities = universities.sample(n=10)

In [9]:
def my_geocoder(row):
    """Returns a geocoded value for a dataframe row"""
    
    try:
        point = geocode(row, provider="nominatim").geometry.iloc[0]
        return pd.Series({'Latitude': point.y, 'Longitude': point.x, 'geometry': point})
    except:
        return None

In [15]:
res = my_geocoder("Eiffel Tower")
res

In [23]:
universities[['Latitude', 'Longitude', 'geometry']] = universities.apply(lambda x: my_geocoder(x['Name']), axis=1)
universities.head()

,Name,Latitude,Longitude,geometry
22,Sorbonne University,48.846900,2.357449,POINT (2.35744865311587 48.8468997)
11,Paris Sciences et Lettres – PSL Research Unive...,NaN,NaN,None
27,University of Warwick,52.379313,-1.560425,POINT (-1.5604252 52.3793131)
39,École Polytechnique,48.847246,2.349096,POINT (2.34909643900866 48.8472458)
71,Scuola Superiore Sant’Anna,43.720608,10.402644,POINT (10.4026444 43.7206078)


In [34]:
print("{}% of the universities we geocoded!".format((1 - universities['Latitude'].isna().sum() / len(universities)) * 100 ))

70.0% of the universities we geocoded!


In [41]:
universities = universities.loc[~np.isnan(universities['Latitude'])]
universities.head()

,Name,Latitude,Longitude,geometry
22,Sorbonne University,48.846900,2.357449,POINT (2.35744865311587 48.8468997)
27,University of Warwick,52.379313,-1.560425,POINT (-1.5604252 52.3793131)
39,École Polytechnique,48.847246,2.349096,POINT (2.34909643900866 48.8472458)
71,Scuola Superiore Sant’Anna,43.720608,10.402644,POINT (10.4026444 43.7206078)
31,Charité - Universitätsmedizin Berlin,52.525851,13.377740,POINT (13.3777395779327 52.52585125)


In [43]:
universities = gpd.GeoDataFrame(universities, geometry=universities.geometry)
universities.crs = {'init': 'epsg:4326'}
universities.head()

,Name,Latitude,Longitude,geometry
22,Sorbonne University,48.846900,2.357449,POINT (2.35745 48.84690)
27,University of Warwick,52.379313,-1.560425,POINT (-1.56043 52.37931)
39,École Polytechnique,48.847246,2.349096,POINT (2.34910 48.84725)
71,Scuola Superiore Sant’Anna,43.720608,10.402644,POINT (10.40264 43.72061)
31,Charité - Universitätsmedizin Berlin,52.525851,13.377740,POINT (13.37774 52.52585)


In [44]:
type(universities)

geopandas.geodataframe.GeoDataFrame

In [49]:
# Create a map
m = folium.Map(location=[54, 15], tiles='openstreetmap', zoom_start=2)

# Add points to map
for ids, row in universities.iterrows():
    Marker(location=[row['Latitude'], row['Longitude']], popup=row['Name']).add_to(m)

# Display the map
embed_map(m, 'm.html')